In [ ]:
# modules
import warnings, os
import pandas as pd
from pandasgui import show

# Custom classes
from DFInfo import DataFrameInfo
from Connector import RDSDatabaseConnector
from GraphPlotter import Plotter
from DFTransform import DataFrameTransform
from DTransform import DataTransform


In [ ]:
# to stop spam of deprecated feature
warnings.simplefilter(action="ignore", category=FutureWarning) 

#creates all the classes
rdsdbc = RDSDatabaseConnector()
dtransformer = DataTransform()
dtinfo = DataFrameInfo()
dftransformer = DataFrameTransform()
plotter = Plotter()

In [ ]:
available_list = ["policy_code"]
string_list = ["policy_code", "term"]
strip_list = ["term"]
qualitative_list = ["purpose", "grade", "sub_grade","home_ownership", "verification_status", "loan_status", "payment_plan", "application_type", "employment_length"]
ignore_transform_list = ["id", "member_id"]

In [ ]:
dataframe = rdsdbc.download_df()
#connects to online server, downloads the database, and stores in a file called loan_payments.csv

In [ ]:
show(dataframe) # original untouched dataframe

In [ ]:
#calls the main function in each data analysis class
dtransformer.set_available_list(available_list)
dtransformer.set_string_list(string_list)
dtransformer.set_strip_list(strip_list)
dtransformer.set_qualitative_list(qualitative_list)

dataframe = dtransformer.call_all_cleaners(dataframe)
####dataframe = dtinfo.call_all_information(dataframe)

In [ ]:
#calls the plotter and visualises the null value removal
nulls_before = dftransformer.count_nulls(dataframe)
dataframe = dftransformer.drop_columns(dataframe)
dataframe = dftransformer.impute_columns(dataframe)
nulls_after = dftransformer.count_nulls(dataframe)

plotter.plot_nulls_before_after(nulls_before, nulls_after)

In [ ]:
skewed_columns = dtransformer.get_skewed_columns(dataframe, qualitative_list, 0.5)
skewed_dataframe = dtransformer.get_skewed_dataframe(dataframe, skewed_columns)
unskewed_dataframe = dftransformer.remove_skewness(dataframe, qualitative_list)

plotter.plot_skewed_vs_unskewed_graph(skewed_dataframe.skew(), unskewed_dataframe.skew())

In [ ]:
combined_df = dftransformer.merge_dataframes(dataframe, unskewed_dataframe)
combined_df.to_csv(os.path.join("script_data", "combined_dataframe.csv"), index=False)

In [ ]:

show(combined_df)